In [4]:
# Import Required Libraries
import pandas as pd
import joblib
import os
from sklearn.preprocessing import StandardScaler

# Load the test dataset (raw format)
test_df = pd.read_csv("/content/drive/MyDrive/Data Science and ML course/Titanic_Dataset/test.csv")

# Feature Engineering (same as training)
# Extract Title from Name
test_df["Title"] = test_df["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
test_df["Title"] = test_df["Title"].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr',
                                             'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test_df["Title"] = test_df["Title"].replace('Mlle', 'Miss')
test_df["Title"] = test_df["Title"].replace('Ms', 'Miss')
test_df["Title"] = test_df["Title"].replace('Mme', 'Mrs')

# Fill missing values
test_df["Age"].fillna(test_df["Age"].median(), inplace=True)
test_df["Fare"].fillna(test_df["Fare"].median(), inplace=True)
test_df["Embarked"].fillna(test_df["Embarked"].mode()[0], inplace=True)

# Create new features
test_df["FamilySize"] = test_df["SibSp"] + test_df["Parch"] + 1
test_df["IsAlone"] = 0
test_df.loc[test_df["FamilySize"] == 1, "IsAlone"] = 1

# Drop unused columns
drop_cols = ["PassengerId", "Name", "Ticket", "Cabin"]
test_ids = test_df["PassengerId"]
test_df.drop(drop_cols, axis=1, inplace=True)

# Convert categorical features
test_df = pd.get_dummies(test_df, columns=["Sex", "Embarked", "Title"], drop_first=True)

# Align with training features (handle missing dummies)
# Load training data to match columns
train_df = pd.read_csv("/content/drive/MyDrive/Data Science and ML course/Titanic_Dataset/train.csv")
X_train = train_df.drop("Survived", axis=1)
missing_cols = set(X_train.columns) - set(test_df.columns)
for col in missing_cols:
    test_df[col] = 0
test_df = test_df[X_train.columns]  # Reorder columns

# Load the trained model and scaler
model = joblib.load("/content/drive/MyDrive/Data Science and ML course/Titanic_Dataset/models/best_model_logistic_regression.pkl")
scaler = joblib.load("/content/drive/MyDrive/Data Science and ML course/Titanic_Dataset/models/scaler.pkl")

# Scale the test data
X_test_scaled = scaler.transform(test_df)

# 6. Make predictions
predictions = model.predict(X_test_scaled)

# Create submission DataFrame
submission = pd.DataFrame({
    "PassengerId": test_ids,
    "Survived": predictions
})

# Save to CSV
os.makedirs("/content/drive/MyDrive/Data Science and ML course/submissions", exist_ok=True)
submission.to_csv("/content/drive/MyDrive/Data Science and ML course/submissions/titanic_submission.csv", index=False)
print("✅ Submission saved")


✅ Submission saved


<ipython-input-4-3283806147>:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df["Age"].fillna(test_df["Age"].median(), inplace=True)
<ipython-input-4-3283806147>:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 